This code uses trained pytorch models to rank best EEG samples per class

In [1]:
import sys
import numpy as np
import scipy.io
import torch
from matplotlib import pyplot as plt
from torchvision import transforms
sys.path.append('./models/')
sys.path.append('./source/')

import utils as myUtils
from DataTypes import EEGDataset
from CNN_1 import Network

# Check does your computer support using GPU
print("GPU_available={}".format(torch.cuda.is_available()))

GPU_available=True


In [ ]:
# HERE WE USE THE TESTING SET
mat = scipy.io.loadmat('EEG_samples_pruned.mat') # make sure to choose the right sample set
mode = 'Kfold_pruned' # make sure to change the mode to get the correct trained models
method = 'sorting' # This code sorts/choses the 10 best EEG samples
EEG_samples = mat['EEG_samples']
trial_no = 'trial21'
no_participants = 29
model_classes = [0,1] # 1 => Sick Class; 0 => Normal Class
export_matlab = {} # To be exported for matlab analysis

best_none_sick_stack, worst_none_sick_stack = [], [] # best and worst non sick samples
best_sick_stack, worst_sick_stack = [], [] # best and worst sick samples
best_none_sick, best_sick = [], []
worst_none_sick, worst_sick = [], []

for partic_id in range(0,no_participants):

    # initiate arrays
    
    # True positive probability
    sick_probs = []
    none_sick_probs = []

    # Get the dataset
    full_data, train_data, full_data = myUtils.generateTrainTest(EEG_samples, partic_id, normalize = False, LOU = True)

    # Transform the dataset to standard tensor datatype
    data_transform = transforms.Compose([transforms.ToTensor()])
    full_set = EEGDataset(data_set=full_data[0], label_set=full_data[1], transform=data_transform)

    # Loading the trained model
    model = Network(num_classes=2)
    model = model.cuda() # Move the model to the GPU
    state_dict = torch.load('./trained_models/'+trial_no+'/'+mode+'/'+'Arch'+str(partic_id+1)+'.pth')  # Load the model's state dictionary.
    model.load_state_dict(state_dict) # Load the model state into the custom model
    model.eval()  # Set the model to evaluation mode.

    for sample_id in range(0, full_set.__len__()):
        # Run the model on the inputs and get scores
        with torch.no_grad():  # No need to calculate gradients for this step.
            eeg_sample, label = full_set.__getitem__(sample_id)
            eeg_sample = model(eeg_sample.unsqueeze(0).float().cuda())  # Run the model on the EEG samples.

        scores = torch.nn.functional.softmax(eeg_sample[0], dim=0)  # Calculate the softmax to get the scores.
        #print(str(scores)+" | "+str(label[0]))  # Print the scores.

        if(label[0] == 0):
            none_sick_probs.append(scores[0].item())
        else:
            sick_probs.append(scores[1].item())

    # Get the 10 best and worst sick and non-sick sample indexes
    if len(none_sick_probs) != 0:
        best_index_non_sick = myUtils.getSortedIndex(none_sick_probs, 'best')
        worst_index_non_sick = myUtils.getSortedIndex(none_sick_probs, 'worst')
    if len(sick_probs) != 0:
        best_index_sick = myUtils.getSortedIndex(sick_probs, 'best')
        worst_index_sick = myUtils.getSortedIndex(sick_probs, 'worst') 


    #Get Samples
    if len(none_sick_probs) != 0:
        best_none_sick , __ = full_set.__getitem__(best_index_non_sick)
        best_none_sick = np.transpose(np.array(best_none_sick), (2, 0, 1))

        worst_none_sick , __ = full_set.__getitem__(worst_index_non_sick)
        worst_none_sick = np.transpose(np.array(worst_none_sick), (2, 0, 1))
    if len(sick_probs) != 0:
        best_sick , __ = full_set.__getitem__(best_index_sick)
        best_sick = np.transpose(np.array(best_sick), (2, 0, 1))

        worst_sick , __ = full_set.__getitem__(worst_index_sick)
        worst_sick = np.transpose(np.array(worst_sick), (2, 0, 1))

    
            

    #appending the activations of the 2 classes for each participant
    field_name = f'field{partic_id+1}'
    export_matlab[field_name] = { 
        'true_sick' : best_sick, 
        'true_normal' : best_none_sick,
        'false_sick' : worst_sick,
        'false_normal' : worst_none_sick
    } 

    # Clear arrays for next model
    best_sick_stack, best_none_sick_stack = [], []
    best_sick, best_none_sick = [], []
    worst_sick_stack, worst_none_sick_stack = [], []
    worst_sick, worst_none_sick = [], []
    sick_probs, none_sick_probs = [], []

# Save results to directory
scipy.io.savemat('./Results/Plots/'+trial_no+'/'+mode+'/trial6'+'/'+method+'/matrix_jitter_output.mat', {'export_matlab': export_matlab})


In [2]:
# HERE WE USE THE FULL DATASET

mat = scipy.io.loadmat('EEG_samples_roi.mat') # make sure to choose the right sample set
mode = 'Kfold_roi' # make sure to change the mode to get the correct trained models
method = 'sorting' # This code sorts/choses the 10 best EEG samples
EEG_samples = mat['EEG_samples']
trial_no = 'trial21'
no_participants = 29
model_classes = [0,1] # 1 => Sick Class; 0 => Normal Class
export_matlab = {} # To be exported for matlab analysis

best_none_sick_stack, worst_none_sick_stack = [], [] # best and worst non sick samples
best_sick_stack, worst_sick_stack = [], [] # best and worst sick samples
best_none_sick, best_sick = [], []
worst_none_sick, worst_sick = [], []

# Get the dataset
full_data, train_data, full_data = myUtils.generateTrainTest(EEG_samples, 1, normalize = False, LOU = True)

# Indexes to exclude
exclude_index = []

for partic_id in range(0,no_participants):

    # initiate arrays
    
    # True positive probability
    sick_probs = []
    none_sick_probs = []

    
    # Transform the dataset to standard tensor datatype
    data_transform = transforms.Compose([transforms.ToTensor()])
    full_set = EEGDataset(data_set=full_data[0], label_set=full_data[1], transform=data_transform)

    # Loading the trained model
    model = Network(num_classes=2)
    model = model.cuda() # Move the model to the GPU
    state_dict = torch.load('./trained_models/'+trial_no+'/'+mode+'/'+'Arch'+str(partic_id+1)+'.pth')  # Load the model's state dictionary.
    model.load_state_dict(state_dict) # Load the model state into the custom model
    model.eval()  # Set the model to evaluation mode.

    for sample_id in range(0, full_set.__len__()):
        # Run the model on the inputs and get scores
        with torch.no_grad():  # No need to calculate gradients for this step.
            eeg_sample, label = full_set.__getitem__(sample_id)
            eeg_sample = model(eeg_sample.unsqueeze(0).float().cuda())  # Run the model on the EEG samples.

        scores = torch.nn.functional.softmax(eeg_sample[0], dim=0)  # Calculate the softmax to get the scores.
        #print(str(scores)+" | "+str(label[0]))  # Print the scores.

        if(label[0] == 0):
            none_sick_probs.append(scores[0].item())
        else:
            sick_probs.append(scores[1].item())

    # Get the 10 best and worst sick and non-sick sample indexes
    if len(none_sick_probs) != 0:
        best_index_non_sick = myUtils.getSortedIndex(none_sick_probs, 'best')
        worst_index_non_sick = myUtils.getSortedIndex(none_sick_probs, 'worst')
    if len(sick_probs) != 0:
        best_index_sick = myUtils.getSortedIndex(sick_probs, 'best')
        worst_index_sick = myUtils.getSortedIndex(sick_probs, 'worst') 


    #Get Samples
    if len(none_sick_probs) != 0:
        best_none_sick , __ = full_set.__getitem__(best_index_non_sick)
        best_none_sick = np.transpose(np.array(best_none_sick), (2, 0, 1))

        worst_none_sick , __ = full_set.__getitem__(worst_index_non_sick)
        worst_none_sick = np.transpose(np.array(worst_none_sick), (2, 0, 1))
    if len(sick_probs) != 0:
        best_sick , __ = full_set.__getitem__(best_index_sick)
        best_sick = np.transpose(np.array(best_sick), (2, 0, 1))

        worst_sick , __ = full_set.__getitem__(worst_index_sick)
        worst_sick = np.transpose(np.array(worst_sick), (2, 0, 1))

    
            

    #appending the activations of the 2 classes for each participant
    field_name = f'field{partic_id+1}'
    export_matlab[field_name] = { 
        'true_sick' : best_sick, 
        'true_normal' : best_none_sick,
        'false_sick' : worst_sick,
        'false_normal' : worst_none_sick
    } 

    # Clear arrays for next model
    best_sick_stack, best_none_sick_stack = [], []
    best_sick, best_none_sick = [], []
    worst_sick_stack, worst_none_sick_stack = [], []
    worst_sick, worst_none_sick = [], []
    sick_probs, none_sick_probs = [], []

# Save results to directory
scipy.io.savemat('./Results/Plots/'+trial_no+'/'+mode+'/trial6'+'/'+method+'/train_matrix_jitter_output.mat', {'export_matlab': export_matlab})

c:\Users\Abdulsatar\Documents\UBICOMP\Thesis Work\ReverseNeuro\./models\CNN_1.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax8(x)


In [ ]:
# This code is for counting the number of samples in each class category i.e true sick, true non sick, false sick, false non sick
mat = scipy.io.loadmat('EEG_samples_pruned.mat') # make sure to choose the right sample set
mode = 'Kfold_pruned' # make sure to change the mode to get the correct trained models
method = 'sorting' # This code sorts/choses the 10 best EEG samples
EEG_samples = mat['EEG_samples']
trial_no = 'trial21'
no_participants = 29
model_classes = [0,1] # 1 => Sick Class; 0 => Normal Class
export_matlab = {} # To be exported for matlab analysis

best_none_sick_stack, worst_none_sick_stack = 0, 0 # best and worst non sick samples
best_sick_stack, worst_sick_stack = 0, 0 # best and worst sick samples

for partic_id in range(0,no_participants):

    # initiate arrays
    
    # True positive probability
    sick_probs = []
    none_sick_probs = []

    # Get the dataset
    full_data, train_data, full_data = myUtils.generateTrainTest(EEG_samples, partic_id, normalize = False, LOU = True)

    # Transform the dataset to standard tensor datatype
    data_transform = transforms.Compose([transforms.ToTensor()])
    full_set = EEGDataset(data_set=full_data[0], label_set=full_data[1], transform=data_transform)

    # Loading the trained model
    model = Network(num_classes=2)
    model = model.cuda() # Move the model to the GPU
    state_dict = torch.load('./trained_models/'+trial_no+'/'+mode+'/'+'Arch'+str(partic_id+1)+'.pth')  # Load the model's state dictionary.
    model.load_state_dict(state_dict) # Load the model state into the custom model
    model.eval()  # Set the model to evaluation mode.

    for sample_id in range(0, full_set.__len__()):
        # Run the model on the inputs and get scores
        with torch.no_grad():  # No need to calculate gradients for this step.
            eeg_sample, label = full_set.__getitem__(sample_id)
            eeg_sample = model(eeg_sample.unsqueeze(0).float().cuda())  # Run the model on the EEG samples.

        scores = torch.nn.functional.softmax(eeg_sample[0], dim=0)  # Calculate the softmax to get the scores.
        #print(str(scores)+" | "+str(label[0]))  # Print the scores.

        if(label[0] == 0):
            none_sick_probs.append(scores[0].item())
        else:
            sick_probs.append(scores[1].item())

    # Get the 10 best and worst sick and non-sick sample indexes
    if len(none_sick_probs) != 0:
        best_index_non_sick = (np.array(none_sick_probs) > 0.5).sum()
        worst_index_non_sick = (np.array(none_sick_probs) <= 0.5).sum()
    if len(sick_probs) != 0:
        best_index_sick = (np.array(sick_probs) > 0.5).sum()
        worst_index_sick = (np.array(sick_probs) <= 0.5).sum()

    best_none_sick_stack = best_none_sick_stack + best_index_non_sick
    worst_none_sick_stack = worst_none_sick_stack + worst_index_non_sick
    best_sick_stack = best_sick_stack + best_index_sick
    worst_sick_stack = worst_sick_stack + worst_index_sick



print("Lable=> Non Sick, Model => Non Sick: "+ str(best_none_sick_stack))
print("Lable=> Non Sick, Model => Sick: "+ str(worst_none_sick_stack))
print("Label=> Sick, Model=> Sick: "+ str(best_sick_stack))
print("Label=> Sick, Model=> Non Sick: "+ str(worst_sick_stack))
   
